In [11]:
# default_exp models.ResNet

# ResNet

> This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@gmail.com based on:
* Wang, Z., Yan, W., & Oates, T. (2017, May). Time series classification from scratch with deep neural networks: A strong baseline. In 2017 international joint conference on neural networks (IJCNN) (pp. 1578-1585). IEEE.
* Fawaz, H. I., Forestier, G., Weber, J., Idoumghar, L., & Muller, P. A. (2019). Deep learning for time series classification: a review. Data Mining and Knowledge Discovery, 33(4), 917-963.
* Official ResNet TensorFlow implementation: https://github.com/hfawaz/dl-4-tsc
* 👀 kernel filter size 8 has been replaced by 7 (I believe it's a bug since even kernels are not commonly used in practice)

In [12]:
#hide
from nbdev.showdoc import *

In [13]:
#export
import torch
import torch.nn as nn
from timeseries.models.layers import *


class ResBlock(nn.Module):
    def __init__(self, ni, nf, ks=[7, 5, 3]):
        super().__init__()
        self.conv1 = convlayer(ni, nf, ks[0])
        self.conv2 = convlayer(nf, nf, ks[1])
        self.conv3 = convlayer(nf, nf, ks[2])

        # expand channels for the sum if necessary
        self.shortcut = noop if ni == nf else convlayer(ni, nf, ks=1, act_fn=False)
        self.act_fn = nn.ReLU()

    def forward(self, x):
        res = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        sc = self.shortcut(res)
        x += sc
        x = self.act_fn(x)
        return x
    
class ResNet(nn.Module):
    def __init__(self,c_in, c_out):
        super().__init__()
        nf = 64
        self.block1 = ResBlock(c_in, nf, ks=[7, 5, 3])
        self.block2 = ResBlock(nf, nf * 2, ks=[7, 5, 3])
        self.block3 = ResBlock(nf * 2, nf * 2, ks=[7, 5, 3])
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(nf * 2, c_out)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.gap(x).squeeze(-1)
        return self.fc(x)

In [14]:
xb = torch.rand(16, 3, 128)
ResNet(3,2)(xb).shape

torch.Size([16, 2])

In [15]:
#hide
from save_nb import *
from fastai2.data.all import *
from nbdev.export import notebook2script
save_nb()
notebook2script()
test_eq(last_saved() < 10, True)

<IPython.core.display.Javascript object>


Current notebook saved.

Converted 000_utils.ipynb.
Converted 001_core.ipynb.
Converted 002_data.ipynb.
Converted 100_layers.ipynb.
Converted 101_ResNet.ipynb.
Converted index.ipynb.

core.py                        saved          0 s ago
utils.py                       saved          0 s ago
data.py                        saved          0 s ago
ResNet.py                      saved          0 s ago
layers.py                      saved          0 s ago

Correct conversion!
11-03-2020 17:07:25
